<a href="https://colab.research.google.com/github/NongNNew/Project_499/blob/main/Project_499_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN

In [1]:
!git clone https://github.com/NongNNew/Project_499.git

Cloning into 'Project_499'...
remote: Enumerating objects: 751, done.
remote: Counting objects: 100% (303/303), done.
remote: Compressing objects: 100% (299/299), done.
remote: Total 751 (delta 18), reused 280 (delta 4), pack-reused 448
Receiving objects: 100% (751/751), 130.45 MiB | 19.53 MiB/s, done.
Resolving deltas: 100% (46/46), done.
Checking out files: 100% (621/621), done.


In [2]:
import pandas as pd
from glob import glob
import librosa
import librosa.display

In [7]:
fruit_veget = ['กระชาย','กระท้อน','กระเทียม1','กระเทียม2','กระเพรา','กล้วยน้ำว้า','กล้วยหอม',
               'ข้าวโพด','ไข่น้ำ',
               'ตะขบไทย','ตะไคร้',
               'ถั่วฝักยาว','ถั่วลันเตา','ถั่วลิสง',
               'ทับทิม',
               'น้อยหน่า','น้ำเต้า',
               'บวบ','ใบชะพลู1','ใบชะพลู2','ใบบัวบก','ใบแมงลัก','ใบยอ',
               'พริกขี้หนู','พริกสด',
               'ฝรั่ง',
               'พุทรา',
               'มะปราง','มะพลับ','มะละกอ',
               'สับปะรด']

source = {'audio_time':[],
          'sampling_rate':[],
          'feature_extraction':[],
          'label':[],
          'description':[]}

In [8]:
for types in fruit_veget:
    audio_files = glob('/content/Project_499/Record_audio/' + str(types) + '/*.wav')

    for audio in audio_files:
        y,sr = librosa.load(audio,duration=6,offset=0)
        source['description'].append(str(types))
        source['audio_time'].append(y)
        source['sampling_rate'].append(sr)
        source['feature_extraction'].append(librosa.feature.mfcc(y,sr,n_mfcc=13))
        source['label'].append(fruit_veget.index(types))

In [9]:
import re

for i in range(len(source['description'])):
    source['description'][i] = re.findall('[\u0E00-\u0E7F]+',source['description'][i])[0]

In [10]:
pd.DataFrame.from_dict(source)

,audio_time,sampling_rate,feature_extraction,label,description
0,"[-0.0001373291, -0.00010681152, -0.0001373291,...",22050,"[[-641.21436, -640.2155, -639.54114, -638.4883...",0,กระชาย
1,"[0.0004119873, 0.00061035156, 0.0005340576, 0....",22050,"[[-625.25085, -626.8685, -629.381, -632.6441, ...",0,กระชาย
2,"[-9.1552734e-05, -0.00012207031, -0.0001373291...",22050,"[[-644.627, -645.0809, -645.80524, -646.2903, ...",0,กระชาย
3,"[-0.00010681152, -0.00010681152, -0.0001525878...",22050,"[[-622.78125, -623.396, -623.63983, -623.7701,...",0,กระชาย
4,"[-0.0002593994, -0.00022888184, -0.0002593994,...",22050,"[[-639.25616, -639.47345, -640.0064, -640.5056...",0,กระชาย
...,...,...,...,...,...
614,"[-0.00018310547, -0.00024414062, -0.0002288818...",22050,"[[-622.7785, -621.6811, -618.9255, -619.49744,...",30,สับปะรด
615,"[-0.00018310547, -0.0002593994, -0.00022888184...",22050,"[[-666.57153, -663.4788, -662.98236, -663.7374...",30,สับปะรด
616,"[0.0, 0.0, 0.0, 3.0517578e-05, -4.5776367e-05,...",22050,"[[-644.81506, -644.89075, -644.7345, -643.6741...",30,สับปะรด
617,"[-0.00010681152, -0.00019836426, -0.0002441406...",22050,"[[-688.85156, -688.6841, -685.51434, -680.2233...",30,สับปะรด


In [11]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np
from sklearn.model_selection import train_test_split

X = np.array(source['feature_extraction'])
y = np.array(source['label'])

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)

model = keras.Sequential()
model.add(keras.layers.Conv2D(64, 5, input_shape=(13, 259, 1), activation='relu'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Conv2D(64, 5, activation='relu',padding='same'))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(len(np.unique(source['label'])), activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.fit(X_train[:,:,:,None], y_train, epochs=100)

Epoch 1/100
15/15 [==============================] - 30s 43ms/step - loss: 23.3215 - accuracy: 0.0345
Epoch 2/100
15/15 [==============================] - 0s 19ms/step - loss: 3.4445 - accuracy: 0.0302
Epoch 3/100
15/15 [==============================] - 0s 18ms/step - loss: 3.4320 - accuracy: 0.0388
Epoch 4/100
15/15 [==============================] - 0s 18ms/step - loss: 3.4312 - accuracy: 0.0453
Epoch 5/100
15/15 [==============================] - 0s 18ms/step - loss: 3.4288 - accuracy: 0.0625
Epoch 6/100
15/15 [==============================] - 0s 18ms/step - loss: 3.4082 - accuracy: 0.0560
Epoch 7/100
15/15 [==============================] - 0s 18ms/step - loss: 3.3805 - accuracy: 0.0560
Epoch 8/100
15/15 [==============================] - 0s 18ms/step - loss: 3.1674 - accuracy: 0.0970
Epoch 9/100
15/15 [==============================] - 0s 18ms/step - loss: 2.8825 - accuracy: 0.1703
Epoch 10/100
15/15 [==============================] - 0s 18ms/step - loss: 2.3626 - accuracy: 0.32

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 9, 255, 64)        1664      
_________________________________________________________________
dropout (Dropout)            (None, 9, 255, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 127, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 127, 64)        102464    
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 127, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 63, 64)         0         
_________________________________________________________________
flatten (Flatten)            (None, 8064)              0

In [13]:
model.evaluate(X_test[:,:,:,None],y_test)

5/5 [==============================] - 0s 68ms/step - loss: 0.2950 - accuracy: 0.9355


[0.29497453570365906, 0.9354838728904724]